In [ ]:
# Import
import os
import pandas as pd
import numpy as np

from pathlib import Path

from CellPacking.tissuegeneration import sheet_init
from CellPacking.dynamics import Compression, AnisotropicLineTension, ShearPlanarGeometry

from tyssue import Sheet
from tyssue import PlanarGeometry
from tyssue.solvers import QSSolver
from tyssue.solvers.viscous import EulerSolver
from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.basic_events import reconnect
from tyssue.dynamics import model_factory, effectors
from tyssue.core.history import HistoryHdf5 


import matplotlib.pyplot as plt
from tyssue.draw import sheet_view
from CellPacking.plot import superimpose_sheet_view
from CellPacking.plot import sheet_view as ply_sheet_view

In [ ]:
SIM_DIR = Path('/home/u2175049/Documents/Simulations_test')
sim_save_dir = SIM_DIR
try:
    os.mkdir(sim_save_dir)
except FileExistsError:
    pass

In [ ]:
# %pdb

# Apical sheet init

In [ ]:
# Number of cells in x and y axis
nx = 20
ny = 20
noise = 0.3
gamma_0 = 0.
phi = np.pi/2

try : 
    apical_sheet, geom = sheet_init(nx, ny, gamma_0, phi, noise)
except IndexError :
    apical_sheet, geom = sheet_init(nx, ny, gamma_0, phi, noise)

    
apical_sheet.get_opposite()
    
apical_sheet.update_specs({"settings":{"dt":0.01,
                                     'threshold_length': 0.01,
                                     'p_4': 0.1,
                                     'p_5p': 0.01,}})

apical_sheet.face_df['prefered_area'] = 1.
apical_sheet.face_df['area_elasticity'] = 1.
apical_sheet.face_df['prefered_perimeter'] = 3*np.sqrt(apical_sheet.face_df['prefered_area'] )
apical_sheet.face_df['perimeter_elasticity'] = 1.
apical_sheet.vert_df['viscosity'] = 1.0
apical_sheet.edge_df['is_active'] = 1

In [ ]:
fig, ax = sheet_view(apical_sheet, **{'edge':
                                    {'color':apical_sheet.edge_df['gamma'],
                                    'colormap':'bwr'}})
ax.set_aspect('equal')
fig.set_size_inches((12, 12))

# QSSolver, Manager & Model definition

We use the QS solver once to start the simulation next with sheet at an equilibrium stage. 

At this stage we don't apply any anisotropic line tension.

In [ ]:
# Solver
solver_qs = QSSolver(with_t1=True, with_t3=False, with_collisions=False)

# Manager
manager = EventManager('face')
# manager.append(reconnect)


# Model
model = model_factory(
    [
#         AnisotropicLineTension, 
        effectors.FaceAreaElasticity,
#         effectors.PerimeterElasticity,
    ])

In [ ]:
for i in range(20):
#     print(i)
    manager.execute(apical_sheet)
    res = solver_qs.find_energy_min(apical_sheet, ShearPlanarGeometry, model, periodic=False, options={"gtol": 1e-8})
    if res.success is False:
        print (res)
    apical_sheet.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(apical_sheet.Nv, 2))
    ShearPlanarGeometry.update_all(apical_sheet)
    manager.update()

In [ ]:
fig, ax = sheet_view(apical_sheet, **{'edge':
                                    {'color':apical_sheet.edge_df['gamma'],
                                    'colormap':'PiYG'}})
ax.set_aspect('equal')
fig.set_size_inches((12, 12))

In [ ]:
from tyssue.draw.plt_draw import plot_forces, plot_scaled_energies
fig, ax = plot_forces(apical_sheet, ShearPlanarGeometry, model, list('xy'), 0.1 )
fig.set_size_inches((20,20))


# Monolayer creation

In [ ]:

from tyssue.generation import extrude 
from tyssue import Monolayer
from CellPacking.dynamics import ShearMonolayerGeometry

geom = ShearMonolayerGeometry

apical_sheet.face_df['z'] = 0.5
apical_sheet.edge_df['z'] = 0.5
apical_sheet.vert_df['z'] = 0.5

extruded = extrude(apical_sheet.datasets, method='translation')
monolayer = Monolayer('mono', extruded)
geom.update_all(monolayer)
monolayer.sanitize(trim_borders=True, order_edges=True)
monolayer.validate()

monolayer.face_df['prefered_area'] = 1.0
monolayer.face_df['area_elasticity'] = 1.0

monolayer.face_df['prefered_perimeter'] = 3 * np.sqrt(monolayer.face_df['prefered_area'])
monolayer.face_df['perimeter_elasticity'] = 1.0

monolayer.update_specs({"settings":{"dt":0.1,
                                     'threshold_length': 0.01,
                                     'p_4': 0.4,
                                     'p_5p': 0.4,
                                      "nrj_norm_factor": 1.0},
                      "cell": {
                                "x": 0.0,
                                "prefered_vol": 1.5,
                                "area": 0.0,
                                "area_elasticity": 0.05,
                                "prefered_area": 6.0,
                                "is_alive": True,
                                "num_faces": 6,
                                "vol": 0.0,
                                "vol_elasticity": 1.0,
                                "z": 0.0,
                                "y": 0.0},
                      "edge": {
                                "dx": 0.0,
                                "srce": 0,
                                "line_tension": 0.0,
                                "face": 0,
                                "dy": 0.0,
                                "ny": 0.0,
                                "nx": 0.0,
                                "prefered_length": 0.8,
                                "border_elasticity": 2.0,
                                "length": 0.0,
                                "nz": 0.0,
                                "cell": 0,
                                "sub_vol": 0.0,
                                "dz": 0.0,
                                "sub_area": 0.0,
                                "trgt": 0},
                      "vert": {
                                "x": 0.0,
                                "is_active": True,
                                "z": 0.0,
                                "y": 0.0},
                      "face": {
                                "x": 0.0,
                                "is_alive": True,
                                "contractility": 0.0,
                                "z": 0.0,
                                "y": 0.0,}
                    })

In [ ]:
fig = superimpose_sheet_view(monolayer.get_sub_sheet('apical'), monolayer.get_sub_sheet('basal'), ['apical', 'basal'])
fig.show()

# Euler solver

In [ ]:
# Model
# model = model_factory(
#     [
#         AnisotropicLineTension,
#         effectors.FaceAreaElasticity,
#         effectors.PerimeterElasticity,
#     ], effectors.FaceAreaElasticity)

model = model_factory(
    [
        AnisotropicLineTension,
#         effectors.LineTension,
        effectors.FaceAreaElasticity,
#         effectors.PerimeterElasticity,
        effectors.CellAreaElasticity,
        effectors.BorderElasticity,
        effectors.CellVolumeElasticity,
    ],
)

# Manager
manager = EventManager('face', track_event=False)

monolayer.get_opposite_faces()
edge_opp_face = monolayer.upcast_face(monolayer.face_df['opposite'])
monolayer.edge_df['is_border']=False
monolayer.edge_df.loc[edge_opp_face[edge_opp_face==-1].index, 'is_border']=True


monolayer.edge_df.loc[monolayer.edge_df['segment']=='basal', 'gamma_0'] = 0.
monolayer.edge_df.loc[monolayer.edge_df['segment']=='lateral', 'gamma_0'] = 0.
monolayer.edge_df['gamma_0'] = pd.to_numeric(monolayer.edge_df['gamma_0'])
monolayer.edge_df['opposite'] = pd.to_numeric(monolayer.edge_df['opposite'])
monolayer.edge_df['phi0'] = pd.to_numeric(monolayer.edge_df['phi0'])
monolayer.edge_df['z'] = pd.to_numeric(monolayer.edge_df['z'])


# monolayer.edge_df['at_x_boundary'] = False
# monolayer.edge_df['at_y_boundary'] = False
# monolayer.edge_df['at_x_boundary'] = pd.to_numeric(monolayer.edge_df['at_x_boundary'])
# monolayer.edge_df['at_y_boundary'] = pd.to_numeric(monolayer.edge_df['at_y_boundary'])

# monolayer.face_df['at_x_boundary'] = pd.to_numeric(monolayer.face_df['at_x_boundary'])
# monolayer.face_df['at_y_boundary'] = pd.to_numeric(monolayer.face_df['at_y_boundary'])

# monolayer.face_df.loc[monolayer.face_df['segment']=='lateral','is_active']=True
# monolayer.face_df['is_active'] = pd.to_numeric(monolayer.face_df['is_active'])

# monolayer.edge_df.loc[monolayer.edge_df['segment']=='lateral','is_active']=True

monolayer.edge_df.loc[monolayer.edge_df['segment']=='basal', 'is_active']=False
monolayer.edge_df['is_active'] = pd.to_numeric(monolayer.edge_df['is_active'])
monolayer.face_df.loc[monolayer.face_df['segment']=='basal', 'is_active']=False
monolayer.face_df['is_active'] = pd.to_numeric(monolayer.face_df['is_active'])

geom.update_all(monolayer)



In [ ]:
# solver_qs = QSSolver(with_t1=True, with_t3=False, with_collisions=False)
# res = solver_qs.find_energy_min(monolayer, geom, model, periodic=False, options={"gtol": 1e-8})

# # manager.append(reconnect)
# for i in range(50):
#     manager.execute(monolayer)
#     res = solver_qs.find_energy_min(monolayer, geom, model, periodic=False, options={"gtol": 1e-8})
# #     print(res.success)
#     monolayer.vert_df[["x", "y"]] += np.random.normal(scale=1e-3, size=(monolayer.Nv, 2))
#     geom.update_all(monolayer)
#     manager.update()

In [ ]:
# History
history = HistoryHdf5(monolayer, 
                      hf5file=os.path.join(sim_save_dir, 'test.hf5'),
                     track_event=False)

# Solver
solver = EulerSolver(monolayer,
                     geom,
                     model,
                     history=history,
                     manager=manager,
                     auto_reconnect=True)

In [ ]:
solver.solve(tf=0.2,
             dt=apical_sheet.settings['dt'],
            
            )


In [ ]:
# sheet = monolayer.get_sub_sheet('basal')
# fig, ax = sheet_view(sheet, **{'edge':
#                                     {'color':sheet.edge_df['gamma'],
#                                     'colormap':'PiYG'}})
# ax.set_aspect('equal')
# fig.set_size_inches((12, 12))

In [ ]:
fig = superimpose_sheet_view(history.retrieve(0).get_sub_sheet('apical'), monolayer.get_sub_sheet('apical'), ['init', 'end'])
fig.show()

In [ ]:
fig = superimpose_sheet_view(history.retrieve(0).get_sub_sheet('basal'), monolayer.get_sub_sheet('basal'), ['init', 'end'])
fig.show()

In [ ]:
fig = superimpose_sheet_view(history.retrieve(0).get_sub_sheet('apical'), history.retrieve(0).get_sub_sheet('basal'), ['apical', 'basal'])
fig.show()

In [ ]:
fig = superimpose_sheet_view(monolayer.get_sub_sheet('apical'), monolayer.get_sub_sheet('basal'), ['apical', 'basal'])
fig.show()

In [ ]:
from tyssue.draw.plt_draw import plot_forces, plot_scaled_energies
fig, ax = plot_forces(monolayer, geom, model, list('xy'), 0.1 )
fig.set_size_inches((20,20))
fig, ax = plot_forces(monolayer, geom, model, list('xz'), 0.1 )
fig.set_size_inches((20,20))

In [ ]:
from tyssue.io.meshes import save_triangular_mesh
save_triangular_mesh('monolayer.vtk', monolayer)

In [ ]:
sheet = history.retrieve(0.5).get_sub_sheet('apical')
fig, ax = sheet_view(sheet, **{'edge':
                                    {'color':sheet.edge_df['gamma'],
                                    'colormap':'bwr'}})
ax.set_aspect('equal')
fig.set_size_inches((12, 12))

In [ ]:
monolayer.face_df[monolayer.face_df['num_sides']==3].index
np.unique(monolayer.edge_df[monolayer.edge_df['face'].isin(monolayer.face_df[monolayer.face_df['num_sides']==3].index)]['cell'])

In [ ]:
dfsdfsdfs

In [ ]:
monolayer.vert_df.loc[monolayer.vert_df['segment']=='apical', 'z']=1
monolayer.vert_df.loc[monolayer.vert_df['segment']=='basal', 'z']=-1

In [ ]:
np.unique(monolayer.edge_df[monolayer.edge_df['srce']==monolayer.vert_df[monolayer.vert_df['segment']=='lateral'].index[0]]['cell'].to_numpy())

In [ ]:
fgfhf

In [ ]:
import meshio

def save_color_cell(filename, eptm, cell_number, coords=list('xyz'), color=[1., 0., 0., 0.1]):
    
    points, faces = monolayer.vertex_mesh(coords=coords, vertex_normals=False)
    triangles_colours=[]
    cells = []
    cpt=0
    for f in faces:
        if cpt in np.unique(eptm.edge_df[eptm.edge_df['cell']==cell_number]['face']):
            cells.append(("triangle", np.array([f])))
            triangles_colours.append([color])
        cpt+=1

    mesh = meshio.Mesh(points, cells, cell_data={"a":triangles_colours})
    meshio.vtk.write(filename, mesh)
    return points, faces
    

save_triangular_mesh('monolayer.vtk', monolayer)
center_cell = 150
monolayer.edge_df[monolayer.edge_df['srce']==monolayer.vert_df[monolayer.vert_df['segment']=='lateral'].index[0]]['cell'].to_numpy()[0] 
points, faces = save_color_cell(str(center_cell)+'.vtk', monolayer, center_cell)
print(center_cell)
for c in monolayer.get_neighborhood(center_cell, 1)['cell'].to_numpy():
    print(c)
    save_color_cell(str(c)+'.vtk', monolayer, c, color=[0., 1., 0., 0.1])

In [ ]:
monolayer.vert_df[monolayer.vert_df['segment']=='lateral']

In [ ]:
np.unique(apical_sheet_init.face_df.num_sides - history.retrieve(0.01).face_df.num_sides, return_counts=True)

In [ ]:
apical_sheet.face_df['color'] = 0
nb_change = apical_sheet_init.face_df.num_sides - apical_sheet.face_df.num_sides
apical_sheet.face_df.loc[nb_change[nb_change==1].index.to_numpy(), 'color'] = 1
apical_sheet.face_df.loc[nb_change[nb_change>2].index.to_numpy(), 'color'] = 2
apical_sheet.face_df.loc[nb_change[nb_change==-1].index.to_numpy(), 'color'] = -1
apical_sheet.face_df.loc[nb_change[nb_change<-2].index.to_numpy(), 'color'] = -2


import plotly.express as px
import plotly.graph_objects as go
import itertools
from tyssue.core.objects import _ordered_edges
from tyssue.draw.plt_draw import _get_lines


a_lines_x, a_lines_y = _get_lines(apical_sheet_init, list('xy'))

b_lines_x, b_lines_y = _get_lines(apical_sheet, list('xy'))

df = pd.DataFrame([np.concatenate((a_lines_x, b_lines_x)),
                   np.concatenate((a_lines_y, b_lines_y)), 
                   np.concatenate((np.repeat('start', len(a_lines_x)), np.repeat('end', len(b_lines_x))))])
df = df.T
df.columns = ['lines_x', 'lines_y', 'tyssue']

fig = px.line(df, x='lines_x', y='lines_y', color='tyssue')


# 2 changes or more
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']>=2].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#EEA000',  #dark orange
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))

# # 1 change
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']==1].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#EEE60C',  #light orange
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))

# # -1 change
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']==-1].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#7CB5EF',  # blue 
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))
    
# # -2 change
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']==-2].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#0566CA',  # dark blue 
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))


fig.update_layout(
    xaxis_range=[-10,10],
    yaxis_range=[-10,10],
    autosize=False,
    width=1000,
    height=1000,
)

fig.show()

In [ ]:
new_e = []
for k, v in history.trackevent.items():
    if 'add_edge' in v:
        if len(v['add_edge']) > 0:
            new_e.append(v['add_edge'])
new_e

In [ ]:
flatten_list = [element for sublist in new_e for element in sublist]
len(flatten_list)/2


In [ ]:
history.trackevent

# TEST